This notebook contains the following calculations:

1. Finding Nearest Image
2. Testing each function is working or not
3. TC for Linear Search (12.48 after execution)
4. Row wise_error calculation
5. Error Graphs (different notebook)

# 1. Finding Nearest Image using lsh.py

In [1]:
# Modified by Evan Stene for CSCI 5702/7702
# modified by swayanshu, 09/25, based on A 

from pyspark.sql import functions as F
from pyspark.sql import types as T
from functools import reduce
from pyspark.sql import SQLContext   #error 2 resolved 


import findspark
import re
import sys
from pyspark import SparkContext, SparkConf
from IPython.display import display
import numpy as np
import random
import time
import pdb
import unittest
from PIL import Image
from pyspark import SparkContext



findspark.init() 
from pyspark.context import SparkContext 
from pyspark.sql.session import SparkSession 
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)               #error1 resolved
spark = SparkSession(sc)

from pyspark.sql import functions as F
from pyspark.sql import types as T
from functools import reduce

def l1(u,v):
    return int(np.sum(np.absolute(np.array(u)-np.array(v))))

class LSH:
    def __init__(self, A, k, L):
        """
        Initializes the LSH object
        A - dataframe to be searched
        k - number of thresholds in each function
        L - number of functions
        """
        # do not edit this function!
        #self.sc = SparkContext()
        self.sc = sc
        self.k = k
        self.L = L
        self.A = A
        self.functions = self.create_functions()
        self.hashed_A = self.hash_data()
        
    # TODO: Implement this
    def l1(self, u, v):
        """
        Finds the L1 distance between two vectors
        u and v are 1-dimensional Row objects
        l1_distance = |x1-x2|+|y1-y2|
        pandas udf in pyspark
        """
        return int(np.sum(np.absolute(np.array(u)-np.array(v))))

    # TODO: Implement this
    def load_data(self, filename):
        """
        Loads the data into a spark DataFrame, where each row corresponds to
        an image patch -- this step is sort of slow.
        Each row in the data is an image, and there are 400 columns.
        """
        try:
            self.A = spark.read.csv(filename)
            print("Data loaded Succesfully")
        except:
            print("can't able to open the file!")
            
        return
        raise NotImplementedError

    # TODO: Implement this
    def create_function(self, dimensions, thresholds):
        """
        Creates a hash function from a list of dimensions and thresholds.
        """
        def f(v):
            return_file = []
            for jdx in range(len(dimensions)):
                return_file.append(1 if v[dimensions[jdx]] >= thresholds[jdx] else 0)
            return return_file
                
        udf_f = F.udf(f,T.ArrayType(T.IntegerType()))
        return udf_f

    def create_functions(self, num_dimensions=400, min_threshold=0, max_threshold=255):             #based on data
        """
        Creates the LSH functions (functions that compute L K-bit hash keys).
        Each function selects k dimensions (i.e. column indices of the image matrix)
        at random, and then chooses a random threshold for each dimension, between 0 and
        255.  For any image, if its value on a given dimension is greater than or equal to
        the randomly chosen threshold, we set that bit to 1.  Each hash function returns
        a length-k bit string of the form "0101010001101001...", and the L hash functions 
        will produce L such bit strings for each image.
        """
        functions = []
        for i in range(self.L):
            dimensions = np.random.randint(low = 0, 
                                    high = num_dimensions,
                                    size = self.k)
            dimensions = [int(x) for x in list(dimensions)]
            thresholds = np.random.randint(low = min_threshold, 
                                    high = max_threshold + 1, 
                                    size = self.k)
            thresholds = [int(x) for x in list(dimensions)]
            functions.append(self.create_function(dimensions, thresholds))
        return functions

    # TODO: Implement this
    def hash_vector(self,v):
        """
        Hashes an individual vector (i.e. image).  This produces an array with L
        entries, where each entry is a string of k bits.
        """
        v1 = spark.createDataFrame([v]).withColumn("vector",F.array(*[F.col(name).cast("float") for name in dfc]))
        
        for idx in range(len(self.functions)):
            v1 = v1.withColumn("_hashed_vector_"+str(idx),self.functions[idx](F.col("vector")))
        return v1.collect()[0]     

    # TODO: Implement this
    def hash_data(self):
        """
        """
        """ apply a function on each row of self.A"""
        """ for each row in my dataframe self.A, return hash_vector(r)"""
        
        # transform the dataframe columns
        dfc = list(self.A.columns)
        temporary_df = self.A.withColumn("vector",F.array(*[F.col(name).cast("float") for name in dfc]))
        
        # defining and applying the udf hash_vector function to our data
        hashed_temporary_df = temporary_df
        for idx in range(len(self.functions)):
             hashed_temporary_df = hashed_temporary_df.withColumn("_hashed_vector_"+str(idx),self.functions[idx](F.col("vector")))
        self.hashed_A = hashed_temporary_df
        return hashed_temporary_df
        raise NotImplementedError

    # TODO: Implement this
    def get_candidates(self,hashed_point, query_index):
        """
        Retrieve all of the points that hash to one of the same buckets 
        as the query point.  Do not do any random sampling (unlike what the first
        part of this problem prescribes).
        Don't retrieve a point if it is the same point as the query point.
        """
        index = query_index
        data_query = hashed_point
        bucket = data_query["_hashed_vector_"+str(index)]
        bucket_data_query = self.hashed_A.filter("CAST(_hashed_vector_"+str(index)+" as string) = '"+str(bucket)+"'")
        bucket_data_query = bucket_data_query.withColumn("query_vector_point",F.array(*[F.lit(x) for x in data_query["vector"]])) 
        L1 = F.udf(l1,T.IntegerType())
        scorepoints = bucket_data_query.filter(F.col("vector")!=F.col("query_vector_point")).withColumn("score",L1("query_vector_point","vector"))
        candidates = scorepoints.sort("score",ascending=False)[["vector","score"]].collect()[:3*self.L]
        return candidates
    
    # TODO: Implement this
    def lsh_search(self,query_index, num_neighbors = 10):
        """
        Run the entire LSH algorithm. Query_Index is just a way to select a point that will be query itself.
        """
        data_query = self.hashed_A.limit(query_index).collect()[-1]
        """
        query_point will be composed of the vector and its resulting hashes with each function in self.functions
        """
        sampleRE_vector = []
        for idx in range(self.L):
            sampleRE_vector.append(self.get_candidates(data_query,idx))
            sqlContext.clearCache()

        initial_results = reduce(lambda x,y:x+y,sampleRE_vector)
        sorted_results = sorted(initial_results,key=lambda x:x["score"],reverse=False)
        return sorted_results[:num_neighbors]

# Plots images at the specified rows and saves them each to files.
def plot(A, row_nums, base_filename):
    for row_num in row_nums:
        patch = np.reshape(A[row_num, :], [20, 20])     #given in lsh.py
        im = Image.fromarray(patch)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(base_filename + "-" + str(row_num) + ".png")

# Finds the nearest neighbors to a given vector, using linear search.
# TODO: Implement this
def linear_search(A, query_index, num_neighbors):
    dataframe = list(A.columns)
    initialdf = A.withColumn("vector",F.array(*[F.col(name).cast("float") for name in dataframe]))
    query_point = initialdf.limit(query_index).collect()[-1]
    # sqlContext.clearCache()
    initialdf = initialdf.withColumn("data_query",F.array(*[F.lit(x) for x in query_point["vector"]]))
    Dfl = F.udf(l1,T.IntegerType())
    sorted_dfn = initialdf.withColumn("score",Dfl("vector","data_query")).sort("score",ascending=False).limit(num_neighbors).collect()
    return sorted_dfn

# Write a function that computes the error measure
# TODO: Implement this
def lsh_error():
    raise NotImplementedError

# TESTS #-------------------------------------------------------------------------------------

class TestLSH(unittest.TestCase):
    def test_l1(self):
        u = np.array([1, 2, 3, 4])
        v = np.array([2, 3, 2, 3])
        self.assertEqual(l1(u, v), 4)

    def test_hash_data(self):
        f1 = lambda v: sum(v)
        f2 = lambda v: sum([x * x for x in v])
        A = np.array([[1, 2, 3], [4, 5, 6]])
        self.assertEqual(f1(A[0,:]), 6)
        self.assertEqual(f2(A[0,:]), 14)

        functions = [f1, f2]
        self.assertTrue(np.array_equal(hash_vector(functions, A[0, :]), np.array([6, 14])))
        self.assertTrue(np.array_equal(hash_data(functions, A), np.array([[6, 14], [15, 77]])))

    ### TODO: Write your tests here (they won't be graded, 
    ### but you may find them helpful)


if __name__ == '__main__':
#    unittest.main() ### TODO: Uncomment this to run tests
    # create an LSH object using lsh = LSH(k=16, L=10)
    Final_Model = LSH(A=spark.read.csv("patches.csv"),k=16,L=10)
    results = Final_Model.lsh_search(1)
   
    """
    Your code here
    """



# Testing if all the functions in the main is working

# Linear Search 

In [2]:
def linear_search(A, query_index, num_neighbors):
    dataframe = list(A.columns)
    initialdf = A.withColumn("vector",F.array(*[F.col(name).cast("float") for name in dataframe]))
    query_point = initialdf.limit(query_index).collect()[-1]
    # sqlContext.clearCache()
    initialdf = initialdf.withColumn("data_query",F.array(*[F.lit(x) for x in query_point["vector"]]))
    Dfl = F.udf(l1,T.IntegerType())
    sorted_dfn = initialdf.withColumn("score",Dfl("vector","data_query")).sort("score",ascending=False).limit(num_neighbors).collect()
    return sorted_dfn

In [3]:
Final_Model = LSH(A=spark.read.csv("patches.csv"),k=16,L=10)
results = Final_Model.lsh_search(1)

In [4]:
Final_Model1 = LSH(A=spark.read.csv("patches.csv"),k=24,L=12)
results1 = Final_Model1.lsh_search(1)                            #for different K and L value

In [5]:
results1

[Row(vector=[23.0, 5.0, 58.0, 117.0, 101.0, 86.0, 104.0, 103.0, 85.0, 37.0, 20.0, 21.0, 6.0, 5.0, 16.0, 10.0, 23.0, 10.0, 1.0, 9.0, 29.0, 17.0, 79.0, 118.0, 99.0, 96.0, 101.0, 98.0, 61.0, 22.0, 20.0, 26.0, 6.0, 7.0, 21.0, 9.0, 39.0, 29.0, 12.0, 4.0, 17.0, 49.0, 102.0, 108.0, 97.0, 104.0, 98.0, 95.0, 39.0, 13.0, 18.0, 26.0, 10.0, 13.0, 24.0, 13.0, 38.0, 45.0, 31.0, 5.0, 12.0, 85.0, 118.0, 101.0, 95.0, 101.0, 96.0, 85.0, 24.0, 13.0, 17.0, 22.0, 17.0, 18.0, 23.0, 18.0, 21.0, 43.0, 39.0, 13.0, 40.0, 113.0, 115.0, 99.0, 99.0, 98.0, 91.0, 51.0, 18.0, 20.0, 17.0, 17.0, 23.0, 23.0, 19.0, 21.0, 15.0, 31.0, 30.0, 15.0, 80.0, 127.0, 102.0, 98.0, 106.0, 99.0, 86.0, 12.0, 17.0, 27.0, 18.0, 14.0, 27.0, 24.0, 14.0, 21.0, 22.0, 23.0, 16.0, 11.0, 116.0, 125.0, 104.0, 96.0, 114.0, 87.0, 33.0, 13.0, 20.0, 19.0, 18.0, 19.0, 20.0, 20.0, 19.0, 17.0, 20.0, 22.0, 31.0, 23.0, 125.0, 104.0, 98.0, 114.0, 105.0, 56.0, 18.0, 15.0, 20.0, 19.0, 18.0, 19.0, 20.0, 20.0, 18.0, 16.0, 15.0, 16.0, 25.0, 20.0, 116.0, 95.0,

In [6]:
results           #for k = 16and L = 10

[Row(vector=[35.0, 35.0, 26.0, 134.0, 187.0, 248.0, 180.0, 63.0, 28.0, 49.0, 65.0, 35.0, 26.0, 26.0, 21.0, 15.0, 16.0, 21.0, 18.0, 11.0, 30.0, 29.0, 39.0, 181.0, 199.0, 237.0, 147.0, 46.0, 36.0, 48.0, 51.0, 39.0, 31.0, 9.0, 7.0, 21.0, 21.0, 9.0, 14.0, 38.0, 34.0, 23.0, 63.0, 209.0, 221.0, 207.0, 98.0, 26.0, 42.0, 44.0, 33.0, 38.0, 16.0, 38.0, 40.0, 14.0, 7.0, 17.0, 34.0, 39.0, 43.0, 20.0, 103.0, 201.0, 242.0, 165.0, 56.0, 18.0, 38.0, 34.0, 22.0, 27.0, 14.0, 15.0, 21.0, 22.0, 16.0, 25.0, 71.0, 119.0, 41.0, 26.0, 158.0, 201.0, 243.0, 117.0, 32.0, 23.0, 26.0, 24.0, 18.0, 13.0, 19.0, 10.0, 10.0, 18.0, 31.0, 57.0, 102.0, 145.0, 27.0, 53.0, 205.0, 227.0, 215.0, 74.0, 20.0, 28.0, 16.0, 17.0, 17.0, 7.0, 9.0, 25.0, 36.0, 45.0, 71.0, 103.0, 106.0, 88.0, 21.0, 108.0, 217.0, 241.0, 162.0, 44.0, 16.0, 26.0, 13.0, 16.0, 14.0, 10.0, 13.0, 21.0, 60.0, 115.0, 133.0, 111.0, 97.0, 104.0, 24.0, 156.0, 206.0, 237.0, 121.0, 29.0, 13.0, 20.0, 14.0, 18.0, 11.0, 17.0, 36.0, 79.0, 121.0, 130.0, 118.0, 109.0, 11

# Linear search using patch dataset

In [7]:
import pandas as pd    #not required
DF = spark.read.csv("patches.csv")

In [23]:
import pickle

Linear_search_test = [100,200,300,400,500,600,700,800,900,1000]         #row wise data slicing
for row in Linear_search_test:
    initial_time = time.time()                 #point number expressed in seconds since the epoch
    LRresults = linear_search(DF,row,10)
    end_time = time.time()
    total_covered_time = end_time - initial_time
    sqlContext.clearCache()
    linearSerach_distance = sum([x["score"] for x in LRresults])
    output = {
        "row":row,
        "time":total_covered_time,
        "NeuralNet":LRresults,
        "distance":linearSerach_distance
    }
    with open("./bdm_assignment_a_lr_"+str(row)+"_LRS.pickle","wb") as f:
        pickle.dump(output,f)

In [24]:
output

{'row': 1000,
 'time': 12.483276844024658,
 'NeuralNet': [Row(_c0='2.550000000000000000e+02', _c1='2.550000000000000000e+02', _c2='2.550000000000000000e+02', _c3='2.550000000000000000e+02', _c4='2.550000000000000000e+02', _c5='2.550000000000000000e+02', _c6='2.550000000000000000e+02', _c7='2.550000000000000000e+02', _c8='2.550000000000000000e+02', _c9='2.550000000000000000e+02', _c10='2.550000000000000000e+02', _c11='2.550000000000000000e+02', _c12='2.550000000000000000e+02', _c13='2.550000000000000000e+02', _c14='2.550000000000000000e+02', _c15='2.550000000000000000e+02', _c16='2.550000000000000000e+02', _c17='2.550000000000000000e+02', _c18='2.550000000000000000e+02', _c19='2.550000000000000000e+02', _c20='2.550000000000000000e+02', _c21='2.550000000000000000e+02', _c22='2.550000000000000000e+02', _c23='2.550000000000000000e+02', _c24='2.550000000000000000e+02', _c25='2.550000000000000000e+02', _c26='2.550000000000000000e+02', _c27='2.550000000000000000e+02', _c28='2.5500000000000000

# Row_wise_Error_calculation

In [53]:
# for row 100
import pickle
with open("./bdm_assignment_a_lr_100_LRS.pickle","rb") as f:
    n1 = pickle.load(f)["distance"]
    
    
numerator = output["distance"]
#  for an specific K and L and number of neighbors
R100_Error = numerator/n1
R100_Error

1.5193963903894598

In [26]:
# for row 200
with open("./bdm_assignment_a_lr_200_LRS.pickle","rb") as f:
    n2 = pickle.load(f)["distance"]
    
R200_Error = numerator/n2
R200_Error

1.398404127477528

In [27]:
with open("./bdm_assignment_a_lr_300_LRS.pickle","rb") as f:
    n3 = pickle.load(f)["distance"]
    
R300_Error = numerator/n3
R300_Error

1.0734793079612013

In [28]:
with open("./bdm_assignment_a_lr_400_LRS.pickle","rb") as f:
    n4 = pickle.load(f)["distance"]
    
R400_Error = numerator/n4
R400_Error

1.269432945755602

In [29]:
with open("./bdm_assignment_a_lr_500_LRS.pickle","rb") as f:
    n5 = pickle.load(f)["distance"]
    
R500_Error = numerator/n5
R500_Error

1.5010620421824237

In [30]:
with open("./bdm_assignment_a_lr_600_LRS.pickle","rb") as f:
    n6 = pickle.load(f)["distance"]
    
R600_Error = numerator/n6
R600_Error

1.213588169448867

In [31]:
with open("./bdm_assignment_a_lr_700_LRS.pickle","rb") as f:
    n7 = pickle.load(f)["distance"]
    
R700_Error = numerator/n7
R700_Error

0.8883810694880162

In [32]:
with open("./bdm_assignment_a_lr_800_LRS.pickle","rb") as f:
    n8 = pickle.load(f)["distance"]
    
R800_Error = numerator/n8
R800_Error

1.1547350649900503

In [33]:
with open("./bdm_assignment_a_lr_900_LRS.pickle","rb") as f:
    n9 = pickle.load(f)["distance"]
    
R900_Error = numerator/n9
R900_Error

0.9980068633426377

In [34]:
with open("./bdm_assignment_a_lr_1000_LRS.pickle","rb") as f:
    n10 = pickle.load(f)["distance"]
    
R1000_Error = numerator/n10
R1000_Error

1.0

# While testing the rows my laptop showed Reboot after every iteration so I implimented the test for individual columns

In [ ]:
import pickle

rows_to_test = [100]  #took single row
k_to_test = [14,16,18,20,22,24,26]
l_to_test = [10,12,14,16,18,20,22]  
for k in k_to_test:
    lsh_model = LSH(A=spark.read.csv("patches.csv"),k=k,L=10)
    for row in rows_to_test:
        start_time = time.time()
        t_results=lsh_model.lsh_search(row,3) # first parameter is query_index, second parameter is num_neighbors
        end_time = time.time()
        total_time = end_time - start_time
        result_dict = {
            "row1to1000":row,
            "time":total_time,
            "ANN_allresults":t_results,
            "distance":sum([x["score"] for x in t_results])
        }
        with open("./assignment01_row_"+str(row)+"_k"+str(k)+".pickle","wb") as f:
            pickle.dump(result_dict,f)
        sqlContext.clearCache()

In [13]:
with open("./assignment01_row_100_nn10.pickle","rb") as f:
    row100 = pickle.load(f)


In [ ]:
import pickle

rows_to_test = [200,300,400,500,600,700,800,900,1000]  #took single row
k_to_test = [14,16,18,20,22,24,26]
l_to_test = [10,12,14,16,18,20,22]  
for k in k_to_test:
    lsh_model = LSH(A=spark.read.csv("patches.csv"),k=k,L=10)
    for row in rows_to_test:
        start_time = time.time()
        t_results=lsh_model.lsh_search(row,3) # first parameter is query_index, second parameter is num_neighbors
        end_time = time.time()
        total_time = end_time - start_time
        result_dict = {
            "row1to1000":row,
            "time":total_time,
            "ANN_allresults":t_results,
            "distance":sum([x["score"] for x in t_results])
        }
        with open("./assignment01_row_"+str(row)+"_k"+str(k)+".pickle","wb") as f:
            pickle.dump(result_dict,f)
        sqlContext.clearCache()

# PLot Nearest Images

In [18]:
# Plots images at the specified rows and saves them each to files.
def plot(A, row_nums, base_filename):
    for row_num in row_nums:
        patch = np.reshape(A[row_num, :], [20, 20])     #given in lsh.py
        im = Image.fromarray(patch)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(base_filename + "-" + str(row_num) + ".png")

In [ ]:
np.array([x["vector"] for x in row100["NN"]]),[1,2,3,4,5,6,7,8,9,10],"./top10images"